In [1]:
import pinocchio as pin
from pinocchio.utils import *

import plotly.express as px

In [2]:
model = pin.buildModelFromUrdf("../robot_utils/panda-model/panda_arm.urdf")
data = model.createData()


In [3]:
q0 = np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0])
v0 = np.zeros(7)
a0 = np.ones(7)*10

model.gravity = pin.Motion.Zero()

lr = 1e-3
loss = []
for i in range(10000):

    _, _, _ = pin.computeRNEADerivatives(model,data,q0,v0,a0)

    loss.append(np.linalg.norm(data.tau))

    dtau_dq = data.dtau_dq # Derivatives of the ID w.r.t. the joint config vector
    dtau_dv = data.dtau_dv # Derivatives of the ID w.r.t. the joint velocity vector
    dtau_da = data.M # Derivatives of the ID w.r.t. the joint acceleration vector

    a0 -= 210*lr*dtau_da.T@data.tau;

    
print(a0)
px.line(loss)

[-0.01520452  0.0782934   0.08704893  0.16177758  1.5216598   0.33336931
 10.08507816]


In [4]:
_, _, _ = pin.computeRNEADerivatives(model,data,np.array([1.0, -1.0, 0.5, -2.0, 1.0, 2.0, -0.5]),np.zeros(7),np.array([10, 10, 10, 10, 10, 10, 10]))
np.set_printoptions(precision=3, suppress=True)
print(dtau_da)


[[ 0.813 -0.8    0.317  0.536  0.04  -0.041 -0.006]
 [-0.8    2.105 -0.409 -1.088 -0.022 -0.033  0.008]
 [ 0.317 -0.409  1.666 -0.018 -0.013 -0.193 -0.006]
 [ 0.536 -1.088 -0.018  1.211  0.052  0.103 -0.007]
 [ 0.04  -0.022 -0.013  0.052  0.03  -0.001  0.001]
 [-0.041 -0.033 -0.193  0.103 -0.001  0.062  0.   ]
 [-0.006  0.008 -0.006 -0.007  0.001  0.     0.006]]


In [7]:


q0 = np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0])
qd0 = q0*5
qdd0 = np.ones(7)*10

pin.forwardKinematics(model,data,q0)
J = pin.computeJointJacobian(model,data,q0,7)
R_bloc = np.zeros((6,6))
R_bloc[0:3, 0:3] = data.oMi[7].rotation
R_bloc[3:, 3:] = data.oMi[7].rotation
J = R_bloc@J

print("Initial config: ", q0)
print("Initial velocity: ", J@qd0)

lr = 1e-3
loss = []
for i in range(2000):

    pin.forwardKinematics(model,data,q0)

    pin.computeForwardKinematicsDerivatives(model, data, q0, qd0, qdd0)
    (dv_dq,dv_dv) = pin.getJointVelocityDerivatives(model,data,7,pin.ReferenceFrame.LOCAL)
    
    J = pin.computeJointJacobian(model,data,q0,7)
    R_bloc = np.zeros((6,6))
    R_bloc[0:3, 0:3] = data.oMi[7].rotation
    R_bloc[3:, 3:] = data.oMi[7].rotation
    # J = R_bloc@J

    loss.append(np.linalg.norm(J@qd0))

    q0 -= lr*dv_dq.T@(J@qd0)

    
print("Final config: ", q0)
print("Final velocity: ", J@qd0)
px.line(loss)


Initial config:  [ 1.  1.  1.  1. -1.  1. -1.]
Initial velocity:  [ 1.078  1.585 -3.51  -1.201  0.781  1.85 ]
Final config:  [ 1.     1.447 -0.241  1.747 -0.997  1.542 -1.   ]
Final velocity:  [-1.972 -0.16  -0.051 -0.124  0.16  -0.31 ]


In [26]:
import pinocchio as pin
import numpy as np

model = pin.buildModelFromUrdf("../robot_utils/panda-model/panda_arm.urdf")
data = model.createData()

qTest = np.array([0.0, 0.0, 0.0, -1.5, 0.0, 1.0, 0.0])
qd = np.array([1, 1, 1, 1, 1, 1, 1])

pin.forwardKinematics(model,data,qTest)
# pin.computeJointJacobians(model,data,qTest)

J = pin.computeJointJacobian(model,data,qTest,7)

R_bloc = np.zeros((6,6))
R_bloc[0:3, 0:3] = data.oMi[7].rotation
R_bloc[3:, 3:] = data.oMi[7].rotation
print(R_bloc@J@qd)

[ 0.35818945  1.12140565 -0.00527273  0.51806945 -1.          1.19315464]


In [ ]:
np.set_printoptions(precision=3, suppress=True)
print(R_bloc)


In [19]:
oMdes = pin.SE3(np.eye(3), np.array([1., 0., 1.]))

qRandom = np.array([0.2, 0.3, -0.1, -1.1, 0.2, 1.4, 0.5])
pin.forwardKinematics(model,data,qRandom)
data.oMi[7]

  R =
  0.934675  -0.354976  -0.019329
 -0.347142  -0.923067   0.165649
-0.0766435  -0.148118  -0.985995
  p =  0.618923 0.0804767  0.757975

In [21]:
T = pin.SE3.Identity()
T.translation = np.array([1, 2,3])
T

  R =
1 0 0
0 1 0
0 0 1
  p = 1 2 3

In [43]:
print("test {:.2f}".format(2.0))

test 2.00
